# 0 导入包

In [25]:
import torch
import cv2 as cv
import os
from torch.utils.data import Dataset
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import torch.nn.functional as F
from torch.nn.utils import weight_norm
import time

# 1  超参数

In [26]:
BATCH_SIZE = 20
EPOCHS = 5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2 载入数据集

In [27]:
class utcntrain(Dataset):#数据集

    def __init__(self):
    
        '''读训练集图片'''
   
        with open ('data/train_train_image.txt','r') as tf:
            train_image_file = tf.read().splitlines()    #这种读取方式和readlines()一样，只是去掉了每一行后面的换行符\n

        with open ('data/train_train_label.txt','r') as tf:
            train_label_file = tf.read().splitlines()    #这种读取方式和readlines()一样，只是去掉了每一行后面的换行符\n


        self.train_image=train_image_file
        self.train_label=train_label_file

        
    def __len__(self):# 返回df的长度
      
        return len(self.train_image)
    
    def __getitem__(self, idx):#idx是索引，根据这个索引返回一个样本
        if idx >= 19097:
            trainimage1 = cv.imread(self.train_image[idx]+'/10.jpg')
            trainimage2 = cv.imread(self.train_image[idx]+'/11.jpg')
            trainimage3 = cv.imread(self.train_image[idx]+'/12.jpg')
            trainimage4 = cv.imread(self.train_image[idx]+'/13.jpg')
            trainimage1 = np.transpose(trainimage1, (2,0,1))
            trainimage2 = np.transpose(trainimage2, (2,0,1))
            trainimage3 = np.transpose(trainimage3, (2,0,1))
            trainimage4 = np.transpose(trainimage4, (2,0,1))
                                      
        else:
            trainimage1 = cv.imread(self.train_image[idx]+'/17.jpg')
            trainimage2 = cv.imread(self.train_image[idx]+'/18.jpg')
            trainimage3 = cv.imread(self.train_image[idx]+'/19.jpg')
            trainimage4 = cv.imread(self.train_image[idx]+'/20.jpg')
            trainimage1 = np.transpose(trainimage1, (2,0,1))
            trainimage2 = np.transpose(trainimage2, (2,0,1))
            trainimage3 = np.transpose(trainimage3, (2,0,1))
            trainimage4 = np.transpose(trainimage4, (2,0,1))
       
        trainimage = np.stack((trainimage1,trainimage2,trainimage3,trainimage4))
        trainimage = trainimage/225
        
        labimage = cv.imread(self.train_label[idx]+'.jpg',0)#读灰度图
        ret,labimage = cv.threshold(labimage, 0, 1, cv.THRESH_BINARY | cv.THRESH_TRIANGLE)#二值化
        labimage = np.array([labimage])
        #labimage = np.transpose(labimage, (2,0,1))
        return torch.from_numpy(trainimage).float(),torch.from_numpy(labimage).float() #读一张图为numpy,返回一个列表里两个tensor。

In [28]:
class utcnval(Dataset):#数据集

    def __init__(self):
    
        '''读训练集图片'''
   
        with open ('data/train_val_image.txt','r') as tf:
            train_image_file = tf.read().splitlines()    #这种读取方式和readlines()一样，只是去掉了每一行后面的换行符\n

        with open ('data/train_val_label.txt','r') as tf:
            train_label_file = tf.read().splitlines()    #这种读取方式和readlines()一样，只是去掉了每一行后面的换行符\n


        self.train_image=train_image_file
        self.train_label=train_label_file

        
    def __len__(self):# 返回df的长度
      
        return len(self.train_image)
    
    def __getitem__(self, idx):#idx是索引，根据这个索引返回一个样本
        if idx >= 19097:
            trainimage1 = cv.imread(self.train_image[idx]+'/10.jpg')
            trainimage2 = cv.imread(self.train_image[idx]+'/11.jpg')
            trainimage3 = cv.imread(self.train_image[idx]+'/12.jpg')
            trainimage4 = cv.imread(self.train_image[idx]+'/13.jpg')
            trainimage1 = np.transpose(trainimage1, (2,0,1))
            trainimage2 = np.transpose(trainimage2, (2,0,1))
            trainimage3 = np.transpose(trainimage3, (2,0,1))
            trainimage4 = np.transpose(trainimage4, (2,0,1))
                                      
        else:
            trainimage1 = cv.imread(self.train_image[idx]+'/17.jpg')
            trainimage2 = cv.imread(self.train_image[idx]+'/18.jpg')
            trainimage3 = cv.imread(self.train_image[idx]+'/19.jpg')
            trainimage4 = cv.imread(self.train_image[idx]+'/20.jpg')
            trainimage1 = np.transpose(trainimage1, (2,0,1))
            trainimage2 = np.transpose(trainimage2, (2,0,1))
            trainimage3 = np.transpose(trainimage3, (2,0,1))
            trainimage4 = np.transpose(trainimage4, (2,0,1))
       
        trainimage = np.stack((trainimage1,trainimage2,trainimage3,trainimage4))
        trainimage = trainimage/225
        
        labimage = cv.imread(self.train_label[idx]+'.jpg',0)#读灰度图
        ret,labimage = cv.threshold(labimage, 0, 1, cv.THRESH_BINARY | cv.THRESH_TRIANGLE)#二值化
        labimage = np.array([labimage])
        #labimage = np.transpose(labimage, (2,0,1))
        return torch.from_numpy(trainimage).float(),torch.from_numpy(labimage).float() #读一张图为numpy,返回一个列表里两个tensor

载入train_loader

In [29]:
traindata = utcntrain()#参数为路径
train_loader = torch.utils.data.DataLoader(traindata, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
valdata = utcnval()#参数为路径
val_loader = torch.utils.data.DataLoader(valdata, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

------Test code start------

for batch_idx, (data, target) in enumerate(train_loader):
    data, target
    break

------Test code end------

# 3 Model

## 3.1 TCN部分

### 3.1.1 裁剪多余的Padding

In [30]:
class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size[0]

    def forward(self, x):

        x=x[:, :, :-self.chomp_size].contiguous()

        return x

### 3.1.2 TCN主体（采用3d卷积+残差）

In [31]:
    
class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs):

        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv3d(n_inputs, n_outputs, 
                                            kernel_size=(2,3,3), stride=1, padding=(0,1,1), dilation=(1,1,1), bias=True))
        #self.chomp1 = Chomp1d([1,1,1])  
        self.relu1 = nn.ReLU()

        self.conv2 = weight_norm(nn.Conv3d(n_inputs, n_outputs, 
                                            kernel_size=(2,3,3), stride=1, padding=(0,1,1), dilation=(2,1,1), bias=True))
        #self.chomp2 = Chomp1d([2,1,1])  
        self.relu2 = nn.ReLU()
        
        self.net = nn.Sequential(self.conv1, self.relu1,
                                self.conv2,  self.relu2)
        
        
        #self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1,
         #                       self.conv2, self.chomp2, self.relu2)
        
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):

        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        

    def forward(self, x):
        
        out = self.net(x)
       
        res = x 
       
        
        return self.relu(out + res)

## 3.2 UNet部分

In [32]:
class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_channels // 2, in_channels // 2, kernel_size=2, stride=2)

        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = torch.tensor([x2.size()[2] - x1.size()[2]])
        diffX = torch.tensor([x2.size()[3] - x1.size()[3]])

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])

        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        
    def forward(self, x):
        return self.conv(x)

## 3.3整合（UNet下采样+TCN+UNet上采样）

In [33]:
class UTCN(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(UTCN, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 512)
        self.up1 = Up(1024, 256, bilinear)
        self.up2 = Up(512, 128, bilinear)
        self.up3 = Up(256, 64, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)
 
        self.tcn = TemporalBlock(512,512) #TCN
    
    def forward(self, x):
        xTCN = []
        for i in range(0,4):
            if i==3:
                x1 = self.inc(x[i])
                x2 = self.down1(x1)
                x3 = self.down2(x2)
                x4 = self.down3(x3)
                x5 = self.down4(x4)
                xTCN.append(x5)
            else:
                xd = self.inc(x[i])
                xd = self.down1(xd)
                xd = self.down2(xd)
                xd = self.down3(xd)
                xd = self.down4(xd)
                xTCN.append(xd)
        xTCN=torch.stack((xTCN[0],xTCN[1],xTCN[2],xTCN[3])) #xTCN.shape()=(D,batch_size,C,H,W)
        
        xTCN=xTCN.permute(1,2,0,3,4)  #输入TCN时，变成（batch_size,C,D,H,W)
        
        xTCN_output = self.tcn(xTCN)
        
        xTCN_output = xTCN_output.permute(2,0,1,3,4) #改回(D,batch_size,C,H,W)
        
        xTCN_output = xTCN_output[3]    #只有最后一个有用
        
        x = self.up1(xTCN_output, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        logits = torch.sigmoid(logits)
        return logits

# 4 损失函数

## 4.1 FocalLoss

In [35]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2, weight=None, ignore_index=255):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.weight = weight
        self.ignore_index = ignore_index
        self.bce_fn = nn.BCEWithLogitsLoss(weight=self.weight)

    def forward(self, preds, labels):
        if self.ignore_index is not None:
            mask = labels != self.ignore_index
            labels = labels[mask]
            preds = preds[mask]

        logpt = -self.bce_fn(preds, labels)
        pt = torch.exp(logpt)
        loss = -((1 - pt) ** self.gamma) * self.alpha * logpt
        return loss

## 4.2 CrossEntropyLoss

class_weight = [0.02, 1.02]
class_weight = torch.Tensor(class_weight)
criterion = torch.nn.CrossEntropyLoss(weight=class_weight).to(DEVICE)

## 4.3 DiceLoss

In [10]:
class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()
 
    def forward(self, logits, targets):
        num = targets.size(0)
        smooth = 0
        
        m1 = logits.view(num, -1)
        m2 = targets.view(num, -1)

        intersection = (m1 * m2)
 
        score = 2. * (intersection.sum(1) + smooth) / (m1.sum(1) + m2.sum(1) + smooth)
        score = 1 - score.sum() / num
        return score



# 5 训练

## 5.1 初始化

In [46]:
model = UTCN(3,1).to(DEVICE)
model = torch.nn.DataParallel(model)
#optimizer = optim.Adam(model.parameters(),lr=0.005, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum=0.9)

In [47]:
criterion = FocalLoss()

In [48]:
model.load_state_dict(torch.load('utcn5.pt'))

<All keys matched successfully>

## 5.2 定义训练函数

In [49]:
def train(model, device, train_loader, optimizer, epoch, criterion):
    model.train()
    since = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()        
        data = data.permute(1,0,2,3,4)#(D,B,C,W,H)
        output = model(data)
        lossfn = criterion 
        loss = lossfn(output,target)
        loss.backward()
        
        optimizer.step()
        if(batch_idx+1)%10 == 0: 
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * BATCH_SIZE, len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            with open('lossdata.txt','a') as f:    #设置文件对象
                f.write('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * BATCH_SIZE, len(train_loader.dataset),
                100. * batch_idx/ len(train_loader), loss.item())+'\n')
                
    time_elapsed = time.time() - since
    print('Train Epoch: {} complete in {:.0f}m {:.0f}s'.format(epoch,
        time_elapsed // 60, time_elapsed % 60))

## 5.3 验证函数

In [50]:
def val(model, device, val_loader, criterion):
    model.eval()
    since = time.time()
    loss_sum=[]
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(val_loader):
            data, target = data.to(device), target.to(device)
            
           
            
            data = data.permute(1,0,2,3,4)#(D,B,C,W,H)
            output = model(data)
            lossfn = criterion 
            loss = lossfn(output,target)
            
            loss_sum.append(loss.item())

            if(batch_idx+1)%10== 0: 
                print('Trainval Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * BATCH_SIZE, len(val_loader.dataset),
                    100. * batch_idx / len(val_loader), loss.item()))
                
    time_elapsed = time.time() - since
    print('Train Epoch: {} complete in {:.0f}m {:.0f}s,lossave = {}'.format(epoch,
        time_elapsed // 60, time_elapsed % 60,sum(loss_sum)/len(loss_sum)))

val(model, DEVICE, val_loader, criterion)

## 5.3 训练函数

In [ ]:
for epoch in range(1, EPOCHS + 1):
    train(model, DEVICE, train_loader, optimizer, epoch,criterion)
    val(model, DEVICE, val_loader, criterion)
    
torch.save(model.state_dict(),'utcn5.pt')

Train Epoch: 1 [180/34192 (1%)]	Loss: 0.043936
Train Epoch: 1 [380/34192 (1%)]	Loss: 0.043919
Train Epoch: 1 [580/34192 (2%)]	Loss: 0.043656
Train Epoch: 1 [780/34192 (2%)]	Loss: 0.043904
Train Epoch: 1 [980/34192 (3%)]	Loss: 0.043887
Train Epoch: 1 [1180/34192 (3%)]	Loss: 0.044208
Train Epoch: 1 [1380/34192 (4%)]	Loss: 0.044052
Train Epoch: 1 [1580/34192 (5%)]	Loss: 0.044033
Train Epoch: 1 [1780/34192 (5%)]	Loss: 0.043667
Train Epoch: 1 [1980/34192 (6%)]	Loss: 0.044026
Train Epoch: 1 [2180/34192 (6%)]	Loss: 0.043987
Train Epoch: 1 [2380/34192 (7%)]	Loss: 0.043752
Train Epoch: 1 [2580/34192 (8%)]	Loss: 0.043955
Train Epoch: 1 [2780/34192 (8%)]	Loss: 0.044045
Train Epoch: 1 [2980/34192 (9%)]	Loss: 0.043804
Train Epoch: 1 [3180/34192 (9%)]	Loss: 0.044107
Train Epoch: 1 [3380/34192 (10%)]	Loss: 0.043868
Train Epoch: 1 [3580/34192 (10%)]	Loss: 0.043667
Train Epoch: 1 [3780/34192 (11%)]	Loss: 0.044171
Train Epoch: 1 [3980/34192 (12%)]	Loss: 0.044026
Train Epoch: 1 [4180/34192 (12%)]	Loss: 0

In [43]:
torch.save(model.state_dict(),'utcn5.pt')

# 6 调试

In [34]:
model.load_state_dict(torch.load('utcn3.pt',map_location=torch.device('cpu')))

<All keys matched successfully>

In [35]:
imgtest1 = cv.imread('1.jpg')
imgtest2 = cv.imread('2.jpg')
imgtest3 = cv.imread('3.jpg')
imgtest4 = cv.imread('4.jpg')
imgtest1 = np.transpose(imgtest1, (2,0,1))
imgtest2 = np.transpose(imgtest2, (2,0,1))
imgtest3 = np.transpose(imgtest3, (2,0,1))
imgtest4 = np.transpose(imgtest4, (2,0,1))

imgtest = np.stack((imgtest1,imgtest2,imgtest3,imgtest4))
imgtest = imgtest[np.newaxis,:,:,:,:]
#imgtest = torch.from_numpy(imgtest).float()

In [36]:
imgtest.shape

(1, 4, 3, 128, 256)

In [37]:
imgtest = imgtest.transpose(1,0,2,3,4)
imgtest.shape

(4, 1, 3, 128, 256)

In [38]:
imgtest = torch.from_numpy(imgtest).float()

In [39]:
output = model(imgtest)

In [40]:
output[0].size()

torch.Size([1, 128, 256])

In [41]:
output1 = output[0][0]
#output1=target[0][0]

In [56]:
zero = torch.zeros_like(output1)
ones = torch.ones_like(output1)*255
door = 0.004
output2 = torch.where(output1 > door, ones,output1)
output2 = torch.where(output2 <= door, zero,output2)
output2 = output2.detach().numpy()
output2=np.array(output2,dtype='uint8') 
output2.shape

(128, 256)

In [57]:
cv.imwrite('output1.jpg',output2)

True

In [231]:
a=data[0][3]
a=np.array(a,dtype='uint8') 
a = a.transpose(1,2,0)

In [232]:
cv.imwrite('outputt.jpg',a)

True

In [87]:
data

tensor([[[[[0.8267, 0.9467, 0.9422,  ..., 0.2622, 0.2711, 0.2622],
           [0.8222, 0.9467, 0.9422,  ..., 0.2578, 0.2711, 0.2578],
           [0.8222, 0.9467, 0.9378,  ..., 0.2578, 0.2667, 0.2578],
           ...,
           [0.5422, 0.5333, 0.5244,  ..., 0.6622, 0.6622, 0.6622],
           [0.5378, 0.5289, 0.5200,  ..., 0.6622, 0.6622, 0.6622],
           [0.5378, 0.5289, 0.5200,  ..., 0.6622, 0.6622, 0.6622]],

          [[0.7467, 0.8667, 0.8667,  ..., 0.2489, 0.2489, 0.2400],
           [0.7422, 0.8667, 0.8667,  ..., 0.2444, 0.2489, 0.2356],
           [0.7422, 0.8667, 0.8622,  ..., 0.2444, 0.2444, 0.2356],
           ...,
           [0.5111, 0.5067, 0.4978,  ..., 0.6444, 0.6444, 0.6444],
           [0.5067, 0.5022, 0.4933,  ..., 0.6444, 0.6444, 0.6444],
           [0.5067, 0.5022, 0.4933,  ..., 0.6444, 0.6444, 0.6444]],

          [[0.6978, 0.8178, 0.8089,  ..., 0.2578, 0.2578, 0.2489],
           [0.6933, 0.8178, 0.8089,  ..., 0.2533, 0.2578, 0.2444],
           [0.6933, 0.8178